In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/home/mrzaizai2k/code_Bao/LLM-with-RAG/notebook


In [2]:
import pandas as pd
from datasets import load_dataset, ClassLabel

from setfit import SetFitModel, Trainer, TrainingArguments
from src.utils import config_parser

In [3]:
config = config_parser(data_config_path = 'config/gpt_routing_train_config.yaml')
dataset_name = config['dataset_name']
sentence_model = config['sentence_model']
text_col_name = config['text_col_name']
label_col_name = config['label_col_name']

In [4]:
# Assuming the loaded dataset is named "datasets"
datasets = load_dataset(dataset_name, split="train", cache_dir=True)

# Assuming the loaded dataset is named "datasets"
new_features = datasets.features.copy()  # Get dataset features
new_features[label_col_name] = ClassLabel(num_classes = len(datasets.unique(label_col_name)), names=datasets.unique(label_col_name))  # Extract unique labels and cast
datasets = datasets.cast(new_features)
ddict  = datasets.train_test_split(test_size=config['test_size'], stratify_by_column=label_col_name, shuffle =True)
train_ds, val_ds = ddict["train"], ddict["test"]
ddict_val_test  = val_ds.train_test_split(test_size=0.5, stratify_by_column=label_col_name, shuffle =True)
val_ds, test_ds = ddict_val_test["train"], ddict_val_test["test"]

Casting the dataset:   0%|          | 0/42 [00:00<?, ? examples/s]

In [5]:
train_ds

Dataset({
    features: ['ID', 'text', 'Language', 'Class_text', 'label'],
    num_rows: 31
})

In [6]:
val_ds

Dataset({
    features: ['ID', 'text', 'Language', 'Class_text', 'label'],
    num_rows: 5
})

In [7]:
test_ds

Dataset({
    features: ['ID', 'text', 'Language', 'Class_text', 'label'],
    num_rows: 6
})

In [8]:
model = SetFitModel.from_pretrained(
    sentence_model,
    labels=datasets.unique(label_col_name),
)

/home/mrzaizai2k/code_Bao/LLM-with-RAG/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [9]:
args = TrainingArguments(
    batch_size=config['batch_size'],
    num_epochs=config['num_epochs'],
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

args.eval_strategy = args.evaluation_strategy # SetFitpackage error

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    metric="accuracy",
    column_mapping={"text": text_col_name, "label": label_col_name}  # Map dataset columns to text/label expected by trainer
)
trainer.train()


Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/31 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 512
  Batch size = 4
  Num epochs = 4
  Total optimization steps = 512


Epoch,Training Loss,Validation Loss,Embedding Loss,Rate
1,No log,No log,0.232800,0.000017
2,No log,No log,0.331500,0.000011
3,No log,No log,0.236400,0.000006
4,No log,No log,0.333300,0.000000


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Loading best SentenceTransformer model from step 128.


In [10]:
metrics = trainer.evaluate(test_ds)
print(metrics)

Applying column mapping to the evaluation dataset
***** Running evaluation *****


{'accuracy': 0.5}


In [11]:
if config['push_to_hub']:
    trainer.push_to_hub(config['huggingface_out_dir'])

model.safetensors:   0%|          | 0.00/539M [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/4.96k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [13]:
preds=model.predict(["who has the pen",
                     "The birthday of Kerger", 
                     "explain in detail the karger min cut and it's complexity", 
                     "giải thích chi tiết các nguyên lý của kiểm thử phần mềm", 
                     "Độ phức tạp của thuật toán karger min cut là gì và so sánh với các thuật toán cùng loại khác"])
print("Prediction:", preds)

Prediction: [0, 0, 1, 0, 1]
